In [1]:
import requests
import urllib.parse

from datetime import datetime, timedelta
from flask import Flask, redirect

In [2]:
# creating a key for flask 
app = Flask(__name__)
app.secret_key = 'password' 

In [3]:
# Key access information 

CLIENT_ID = '4db82491748043009a9d79b9a2d6739d' 
CLIENT_SECRET = '890096722afc4bf4ae030f8ddaea6830' 
REDIRECT_URI = 'http://localhost:5000/callback'

AUTH_URL = 'https://accounts.spotify.com/authorize'
TOKEN_URL = 'https://accounts.spotify.com/api/token'
API_BASE_URL = 'https://spi.spotify.com/v1'

In [4]:
@app.route('/')
def index(): 
    return "Welcome to my Spotify App <a href='/login'>Login with Spotify</a>"

# redirecting into spotify's webpage 
@app.route('/login')
def login(): 
    scope = 'user-read-private user-read-email'
    
    # creating the parameters
    params = {
        'client_id': CLIENT_ID, 
        'response_type': 'code', 
        'scope': scope, 
        'redirect_uri': REDIRECT_URI, 
        'show_dialog': True 
      
        # If it is false, then it means that the user does not need to log in again. 
        # True: Testing the user to keep logging in -- for testing locally purposes 
        
    }
    
    auth_url = f"{AUTH_URL}?{urllib.parse.urlencode(params)}" 
    
    return redirect(auth_url)

In [8]:
# writing for the callback endpoint 
@app.route('/callback')
def callback(): 
    # if there is an error, makes sure that there is no 
    if 'error' in request.args: 
        return jsonify({"error": request.args['error']}) 
    
    if 'code' in request.args: 
        # building up a request body with data to send to spotify 
        req_body = {
            'code': request.args['code'],
            'grant_type': 'authorization_code', 
            'redirect_uri': REDIRECT_URI, 
            'client_id': CLIENT_ID, 
            'client_secret': CLIENT_SECRET
        }
        
        response = requests.post(TOKEN_URL, data=req_body) 
        token_info = response.json() # token info typically comes back in as a json object
        
        session['access_token'] = token_info['access_token'] # after one day, this token will go away 
        session['refresh_token'] = token_info['refresh_token'] # only lasts for one day 
        session['expires_at'] = datetime.now().timestamp() + token_info['expires_in'] # getting the current datetime and turning it into a timestamp 
        
        return redirect('/playlist') 

In [9]:
@app.route('/playlists')
def get_playlists(): 
    if 'access_token' not in session: 
        return redirect('/login')
    
    # making sure that the access token has not expired yet 
    if datetime.now().timestamp() > session['expires_at']: 
        return redirect('/refresh-token') # if the access token has expired, then it will access a new refresh token in the background
    
    headers = {
        'Authorization': f"Bearer {session['access_token']}" 
        
    }
    
    #storing the result of making a request
    response = requests.get(API_BASE_URL + 'me/playlists', headers=headers) 
    playlists = response.json()
    
    return jsonify(playlists) # returning playlists back to the user 

In [10]:
# refreshing the token 
@app.route('/refresh-token') 
def refresh_token(): 
    if 'refresh_token' not in session: 
        return redirect('/login') 
    
    if datetime.now().timestamp() > session['expires_at']: 
        req_body = {
            'grant_type': 'refresh_token', 
            'refresh_token': session['refresh_token'], 
            'client_id': CLIENT_ID, 
            'client_secret': CLIENT_SECRET
        }
        
        response = requests.post(TOKEN_URL, data=req_body) 
        new_token_info = response.json()
        
        session['access_token'] = new_token_info['access_token'] 
        session['expires_at'] = datetime.now().timestamp() + token_info['expires_in'] 
        
        return redirect('/playlists') 


In [13]:
if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


OSError: [Errno 48] Address already in use